# Libraries

In [1]:
%%time
!pip install langchain langchain_openai langchain_community langchain-chroma langsmith rank_bm25 -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 19.6 MB/s eta 

In [2]:
%%time
!pip install pypdf docx2txt wikipedia pymupdf -qU

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 16.7 MB/s eta 0:00:00
CPU times: user 248 ms, sys: 22.6 ms, total: 271 ms
Wall time: 23.2 s


# Api Keys

In [3]:
# Load all the required api keys
from google.colab import userdata

LANGCHAIN_API_KEY = userdata.get('LANGCHAIN_API_KEY')
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
OPENAI_API_KEY = userdata.get('openai_key')

# Langsmith Tracing

In [4]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-subrata"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY

project_name = "langchain-subrata"

# Text Wrapper

In [16]:
from textwrap import fill
from IPython.display import display, HTML

def print_wrapped(text, width=80):
    print("="*100)
    wrapped_text = fill(text, width=width)
    display(HTML(f"<pre style='white-space:pre-wrap;'>{wrapped_text}</pre>"))
    print("="*100)

# Prettifier

In [86]:
def print_pretty(x):
  print("="*100)
  print()
  print(x)
  print()
  print("="*100)

# Data Loaders

## Wikipedia Loader

In [7]:
def load_from_wikipedia(query, lang="en", load_max_docs=2):
  """
  docs[0].metadata  # meta-information of the Document
  docs[0].page_content[:400]  # a content of the Document
  """
  from langchain_community.document_loaders import WikipediaLoader

  print("Loading from Wikipedia ...")
  docs = WikipediaLoader(query="Accounting", load_max_docs=2).load()
  print("Done!")
  return docs

In [17]:
%%time
docs = load_from_wikipedia("Accounting")
print_pretty(docs)

Loading from Wikipedia ...
Done!

[Document(metadata={'title': 'Accounting', 'summary': 'Accounting, also known as accountancy, is the process of recording and processing information about economic entities, such as businesses and corporations. Accounting measures the results of an organization\'s economic activities and conveys this information to a variety of stakeholders, including investors, creditors, management, and regulators. Practitioners of accounting are known as accountants. The terms "accounting" and "financial reporting" are often used interchangeably.\nAccounting can be divided into several fields including financial accounting, management accounting, tax accounting and cost accounting. Financial accounting focuses on the reporting of an organization\'s financial information, including the preparation of financial statements, to the external users of the information, such as investors, regulators and suppliers. Management accounting focuses on the measurement, analysis a

In [20]:
print_pretty(docs[0].metadata)
print_wrapped(docs[0].page_content)


{'title': 'Accounting', 'summary': 'Accounting, also known as accountancy, is the process of recording and processing information about economic entities, such as businesses and corporations. Accounting measures the results of an organization\'s economic activities and conveys this information to a variety of stakeholders, including investors, creditors, management, and regulators. Practitioners of accounting are known as accountants. The terms "accounting" and "financial reporting" are often used interchangeably.\nAccounting can be divided into several fields including financial accounting, management accounting, tax accounting and cost accounting. Financial accounting focuses on the reporting of an organization\'s financial information, including the preparation of financial statements, to the external users of the information, such as investors, regulators and suppliers. Management accounting focuses on the measurement, analysis and reporting of information for internal use by mana

## Pdf and Docx Loader

In [21]:
def load_document(file):
  import os

  print("Detecting file type ...")
  name, extension = os.path.splitext(file)
  print(f"{extension[1:]} file type detected!")

  print(f"Loading {file} ...")

  if extension == ".pdf":
    from langchain_community.document_loaders import PyPDFLoader

    loader = PyPDFLoader(file)
    pages = loader.load_and_split()

    print(f"{len(pages)} pages parsed successfully!")
    print("Done!")
    return pages

  elif extension == ".docx":
    from langchain_community.document_loaders import Docx2txtLoader

    loader = Docx2txtLoader(file)
    pages = loader.load_and_split()

    print(f"{len(pages)} pages parsed successfully!")
    print("Done!")
    return pages

  else:
    print(f"Unfortunately! {extension} is not supported")

# Load Data

Breaking down the entire document's pages into each individual page aka Document.

In [22]:
%%time
# Breaking down the entire document's pages into each individual page aka Document.
pages = load_document("/content/accounting.pdf")
print_pretty(pages)

Detecting file type ...
pdf file type detected!
Loading /content/accounting.pdf ...
387 pages parsed successfully!
Done!

[Document(metadata={'source': '/content/accounting.pdf', 'page': 0}, page_content='CHAPTER 1 UNIT - 1  1.MEANING AND SCOPE OF ACCOUNTING 1.INTRODUCTION Every individual performs some kind of economic activity. A salaried person gets salary and spends to buy provisions and clothing, for children\'s education, construction of house, etc. A sports club formed by a group of individuals, a business run by an individual or a group of individuals, a company running a business in telecom sector, a local authority like Calcutta Municipal Corporation, Delhi Development Authority, Governments, either Central or State, all are carrying some kind of economic activities. Not necessarily all the economic activities are run for any individual benefit; such economic activities may create social benefit i.e. benefit for the public, at large. Anyway, such economic activities are perfo

In [23]:
# Display the first page
print_wrapped(pages[0].page_content)

In [24]:
# Display the second page
print_wrapped(pages[1].page_content)

# Vector DB → Chroma DB

## Chunk Data Fn
* Chunk each Document Page with Langchain's `RecursiveCharacterTextSplitter`.

In [30]:
def chunk_data(data, chunk_size=256, chunk_overlap=0):
  from langchain_text_splitters import RecursiveCharacterTextSplitter

  print(f"Chunking ...")

  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=chunk_size,
      chunk_overlap=chunk_overlap
  )
  chunks = text_splitter.split_documents(data)

  print(f"Total Chunks: {len(chunks)}.")
  average_chunk_length = sum([len(chunk.page_content) for chunk in chunks]) / len(chunks)
  print(f"Average chunk length for each chunk: {round(average_chunk_length, 2)} characters.")
  print("Done!")

  return chunks

## Chunk Data

In [31]:
%%time
chunks = chunk_data(data=pages, chunk_size=500, chunk_overlap=50)
print_pretty(chunks)

Chunking ...
Total Chunks: 1858.
Average chunk length for each chunk: 441.58 characters.
Done!

[Document(metadata={'source': '/content/accounting.pdf', 'page': 0}, page_content="CHAPTER 1 UNIT - 1  1.MEANING AND SCOPE OF ACCOUNTING 1.INTRODUCTION Every individual performs some kind of economic activity. A salaried person gets salary and spends to buy provisions and clothing, for children's education, construction of house, etc. A sports club formed by a group of individuals, a business run by an individual or a group of individuals, a company running a business in telecom sector, a local authority like Calcutta Municipal Corporation, Delhi Development Authority,"), Document(metadata={'source': '/content/accounting.pdf', 'page': 0}, page_content="Corporation, Delhi Development Authority, Governments, either Central or State, all are carrying some kind of economic activities. Not necessarily all the economic activities are run for any individual benefit; such economic activities may cre

## Chroma DB

In [32]:
%%time
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI Embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    openai_api_key=OPENAI_API_KEY
  )

# Chroma DB
db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="db"
)

print_pretty(db)



CPU times: user 14 s, sys: 559 ms, total: 14.6 s
Wall time: 20.3 s


## Similarity Search Fn

In [34]:
%%time
def similarity_search_with_score(query, k=2):
  results = db.similarity_search_with_score(query=query, k=k, where_document={"$contains": "procedural aspects"})
  sorted_results = sorted(results, key=lambda x: x[1], reverse=True)

  for result in sorted_results:
    print(result[1])
    print_pretty(result[0].page_content)

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 12.2 µs


In [35]:
%%time
query="What are the procedural aspects of accounting?"
similarity_search_with_score(query, k=2)

0.9075726270675659

6.BOOK-KEEPING Book-keeping is an activity concerned with the recording of financial data relating to business operations in a significant and orderly manner. It covers procedural aspects of accounting work and embraces record keeping function. Obviously, book-keeping procedures are governed by the end product, the financial statements. The term 'financial statements' means Profit and Loss Account, Balance Sheet and cash flow statements including Schedules and Notes forming part of Accounts.

0.9075726270675659

6.BOOK-KEEPING Book-keeping is an activity concerned with the recording of financial data relating to business operations in a significant and orderly manner. It covers procedural aspects of accounting work and embraces record keeping function. Obviously, book-keeping procedures are governed by the end product, the financial statements. The term 'financial statements' means Profit and Loss Account, Balance Sheet and cash flow statements including Schedules a

# Retrievers

# Advanced Retrievers

## Context Enrichment Window for Document Retrieval

### Helper Fn

In [36]:
from langchain.document_loaders import  PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain import PromptTemplate
import fitz
from typing import List
from rank_bm25 import BM25Okapi
import asyncio
import random
import textwrap
import numpy as np





def replace_t_with_space(list_of_documents):
    """
    Replaces all tab characters ('\t') with spaces in the page content of each document.

    Args:
        list_of_documents: A list of document objects, each with a 'page_content' attribute.

    Returns:
        The modified list of documents with tab characters replaced by spaces.
    """

    for doc in list_of_documents:
        doc.page_content = doc.page_content.replace('\t', ' ')  # Replace tabs with spaces
    return list_of_documents


def text_wrap(text, width=120):
    """
    Wraps the input text to the specified width.

    Args:
        text (str): The input text to wrap.
        width (int): The width at which to wrap the text.

    Returns:
        str: The wrapped text.
    """
    return textwrap.fill(text, width=width)




def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using OpenAI embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A FAISS vector store containing the encoded book content.
    """

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # Create embeddings and vector store
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore

def encode_from_string(content):
    text_splitter = RecursiveCharacterTextSplitter(
        # Set a really small chunk size, just to show.
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False,
    )
    chunks = text_splitter.create_documents([content])

    for chunk in chunks:
        chunk.metadata['relevance_score'] = 1.0

    embeddings = OpenAIEmbeddings()

    vectorstore = FAISS.from_documents(chunks, embeddings)
    return vectorstore


def retrieve_context_per_question(question, chunks_query_retriever):
    """
    Retrieves relevant context and unique URLs for a given question using the chunks query retriever.

    Args:
        question: The question for which to retrieve context and URLs.

    Returns:
        A tuple containing:
        - A string with the concatenated content of relevant documents.
        - A list of unique URLs from the metadata of the relevant documents.
    """

    # Retrieve relevant documents for the given question
    docs = chunks_query_retriever.get_relevant_documents(question)

    # Concatenate document content
    # context = " ".join(doc.page_content for doc in docs)
    context = [doc.page_content for doc in docs]


    return context

class QuestionAnswerFromContext(BaseModel):
    """
    Model to generate an answer to a query based on a given context.

    Attributes:
        answer_based_on_content (str): The generated answer based on the context.
    """
    answer_based_on_content: str = Field(description="Generates an answer to a query based on a given context.")

def create_question_answer_from_context_chain(llm):

    # Initialize the ChatOpenAI model with specific parameters
    question_answer_from_context_llm = llm

    # Define the prompt template for chain-of-thought reasoning
    question_answer_prompt_template = """
    For the question below, provide a concise but suffice answer based ONLY on the provided context:
    {context}
    Question
    {question}
    """

    # Create a PromptTemplate object with the specified template and input variables
    question_answer_from_context_prompt = PromptTemplate(
        template=question_answer_prompt_template,
        input_variables=["context", "question"],
    )

    # Create a chain by combining the prompt template and the language model
    question_answer_from_context_cot_chain = question_answer_from_context_prompt | question_answer_from_context_llm.with_structured_output(QuestionAnswerFromContext)
    return question_answer_from_context_cot_chain



def answer_question_from_context(question, context, question_answer_from_context_chain):
    """
    Answer a question using the given context by invoking a chain of reasoning.

    Args:
        question: The question to be answered.
        context: The context to be used for answering the question.

    Returns:
        A dictionary containing the answer, context, and question.
    """
    input_data = {
        "question": question,
        "context": context
    }
    print("Answering the question from the retrieved context...")

    output = question_answer_from_context_chain.invoke(input_data)
    answer = output.answer_based_on_content
    return {"answer": answer, "context": context, "question": question}


def show_context(context):
    """
    Display the contents of the provided context list.

    Args:
        context (list): A list of context items to be displayed.

    Prints each context item in the list with a heading indicating its position.
    """
    for i, c in enumerate(context):
        print(f"Context {i+1}:")
        print(c)
        print("\n")


def read_pdf_to_string(path):
    """
    Read a PDF document from the specified path and return its content as a string.

    Args:
        path (str): The file path to the PDF document.

    Returns:
        str: The concatenated text content of all pages in the PDF document.

    The function uses the 'fitz' library (PyMuPDF) to open the PDF document, iterate over each page,
    extract the text content from each page, and append it to a single string.
    """
    # Open the PDF document located at the specified path
    doc = fitz.open(path)
    content = ""
    # Iterate over each page in the document
    for page_num in range(len(doc)):
        # Get the current page
        page = doc[page_num]
        # Extract the text content from the current page and append it to the content string
        content += page.get_text()
    return content



def bm25_retrieval(bm25: BM25Okapi, cleaned_texts: List[str], query: str, k: int = 5) -> List[str]:
    """
    Perform BM25 retrieval and return the top k cleaned text chunks.

    Args:
    bm25 (BM25Okapi): Pre-computed BM25 index.
    cleaned_texts (List[str]): List of cleaned text chunks corresponding to the BM25 index.
    query (str): The query string.
    k (int): The number of text chunks to retrieve.

    Returns:
    List[str]: The top k cleaned text chunks based on BM25 scores.
    """
    # Tokenize the query
    query_tokens = query.split()

    # Get BM25 scores for the query
    bm25_scores = bm25.get_scores(query_tokens)

    # Get the indices of the top k scores
    top_k_indices = np.argsort(bm25_scores)[::-1][:k]

    # Retrieve the top k cleaned text chunks
    top_k_texts = [cleaned_texts[i] for i in top_k_indices]

    return top_k_texts



async def exponential_backoff(attempt):
    """
    Implements exponential backoff with a jitter.

    Args:
        attempt: The current retry attempt number.

    Waits for a period of time before retrying the operation.
    The wait time is calculated as (2^attempt) + a random fraction of a second.
    """
    # Calculate the wait time with exponential backoff and jitter
    wait_time = (2 ** attempt) + random.uniform(0, 1)
    print(f"Rate limit hit. Retrying in {wait_time:.2f} seconds...")

    # Asynchronously sleep for the calculated wait time
    await asyncio.sleep(wait_time)

async def retry_with_exponential_backoff(coroutine, max_retries=5):
    """
    Retries a coroutine using exponential backoff upon encountering a RateLimitError.

    Args:
        coroutine: The coroutine to be executed.
        max_retries: The maximum number of retry attempts.

    Returns:
        The result of the coroutine if successful.

    Raises:
        The last encountered exception if all retry attempts fail.
    """
    for attempt in range(max_retries):
        try:
            # Attempt to execute the coroutine
            return await coroutine
        except RateLimitError as e:
            # If the last attempt also fails, raise the exception
            if attempt == max_retries - 1:
                raise e

            # Wait for an exponential backoff period before retrying
            await exponential_backoff(attempt)

    # If max retries are reached without success, raise an exception
    raise Exception("Max retries reached")

### Pdf Path

In [37]:
path = "/content/accounting.pdf"

### Convert Docx to Strings

In [38]:
def read_pdf_to_string(path):
    """
    Read a PDF document from the specified path and return its content as a string.

    Args:
        path (str): The file path to the PDF document.

    Returns:
        str: The concatenated text content of all pages in the PDF document.

    The function uses the 'fitz' library (PyMuPDF) to open the PDF document, iterate over each page,
    extract the text content from each page, and append it to a single string.
    """
    # Open the PDF document located at the specified path
    doc = fitz.open(path)
    content = ""
    # Iterate over each page in the document
    for page_num in range(len(doc)):
        # Get the current page
        page = doc[page_num]
        # Extract the text content from the current page and append it to the content string
        content += page.get_text()
    return content

In [39]:
%%time
content = read_pdf_to_string(path)
print_pretty(content)


CHAPTER 1 
UNIT - 1  
1.MEANING AND SCOPE OF ACCOUNTING 
1.INTRODUCTION 
Every individual performs some kind of economic activity. A salaried person gets 
salary and spends to buy provisions and clothing, for children's education, 
construction of house, etc. A sports club formed by a group of individuals, a 
business run by an individual or a group of individuals, a company running a 
business in telecom sector, a local authority like Calcutta Municipal Corporation, 
Delhi Development Authority, Governments, either Central or State, all are 
carrying some kind of economic activities. Not necessarily all the economic 
activities are run for any individual benefit; such economic activities may create 
social benefit i.e. benefit for the public, at large. Anyway, such economic activities 
are performed through 'transactions and events'. Transaction is used to mean 'a 
business, performance of an act, an agreement' while event is used to mean 'a 
happening, as a consequence of transactio

### Function to split text into chunks with metadata of the chunk chronological index

In [40]:
from langchain.docstore.document import Document

def split_text_to_chunks_with_indices(text: str, chunk_size: int, chunk_overlap: int) -> List[Document]:
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(Document(page_content=chunk, metadata={"index": len(chunks), "text": text}))
        start += chunk_size - chunk_overlap
    return chunks

### Split our document accordingly

In [41]:
%%time
chunks_size = 200
chunk_overlap = 100
docs = split_text_to_chunks_with_indices(content, chunks_size, chunk_overlap)
type(docs)

CPU times: user 125 ms, sys: 3.98 ms, total: 129 ms
Wall time: 410 ms


list

In [42]:
len(docs)

7710

### Create vector store and retriever

In [ ]:
# embeddings = OpenAIEmbeddings()
# vectorstore = FAISS.from_documents(docs, embeddings)
# chunks_query_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [43]:
# %%time
# from langchain_chroma import Chroma
# from langchain_openai import OpenAIEmbeddings

# # OpenAI Embeddings
# embeddings = OpenAIEmbeddings(
#     model="text-embedding-3-large",
#     openai_api_key=OPENAI_API_KEY
#   )

# # Chroma DB
# db = Chroma.from_documents(
#     documents=docs,
#     embedding=embeddings,
#     persist_directory="advanced-rag-db"
# )

# chunks_query_retriever = db.as_retriever(search_kwargs={"k": 2})
# type(chunks_query_retriever)

## Semantic Chunking

In [84]:
!pip install deepeval -qU

In [83]:
"""
RAG Evaluation Script

This script evaluates the performance of a Retrieval-Augmented Generation (RAG) system
using various metrics from the deepeval library.

Dependencies:
- deepeval
- langchain_openai
- json

Custom modules:
- helper_functions (for RAG-specific operations)
"""

import json
from typing import List, Tuple

from deepeval import evaluate
from deepeval.metrics import GEval, FaithfulnessMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from langchain_openai import ChatOpenAI


os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

def create_deep_eval_test_cases(
    questions: List[str],
    gt_answers: List[str],
    generated_answers: List[str],
    retrieved_documents: List[str]
) -> List[LLMTestCase]:
    """
    Create a list of LLMTestCase objects for evaluation.

    Args:
        questions (List[str]): List of input questions.
        gt_answers (List[str]): List of ground truth answers.
        generated_answers (List[str]): List of generated answers.
        retrieved_documents (List[str]): List of retrieved documents.

    Returns:
        List[LLMTestCase]: List of LLMTestCase objects.
    """
    return [
        LLMTestCase(
            input=question,
            expected_output=gt_answer,
            actual_output=generated_answer,
            retrieval_context=retrieved_document
        )
        for question, gt_answer, generated_answer, retrieved_document in zip(
            questions, gt_answers, generated_answers, retrieved_documents
        )
    ]

# Define evaluation metrics
correctness_metric = GEval(
    name="Correctness",
    model="gpt-4o",
    evaluation_params=[
        LLMTestCaseParams.EXPECTED_OUTPUT,
        LLMTestCaseParams.ACTUAL_OUTPUT
    ],
    evaluation_steps=[
        "Determine whether the actual output is factually correct based on the expected output."
    ],
)

faithfulness_metric = FaithfulnessMetric(
    threshold=0.7,
    model="gpt-4o",
    include_reason=False
)

relevance_metric = ContextualRelevancyMetric(
    threshold=1,
    model="gpt-4o",
    include_reason=True
)

def evaluate_rag(chunks_query_retriever, num_questions: int = 5) -> None:
    """
    Evaluate the RAG system using predefined metrics.

    Args:
        chunks_query_retriever: Function to retrieve context chunks for a given query.
        num_questions (int): Number of questions to evaluate (default: 5).
    """
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=2000, openai_api_key=OPENAI_API_KEY,)
    question_answer_from_context_chain = create_question_answer_from_context_chain(llm)

    # Load questions and answers from JSON file
    q_a_file_name = "../data/q_a.json"
    with open(q_a_file_name, "r", encoding="utf-8") as json_file:
        q_a = json.load(json_file)

    questions = [qa["question"] for qa in q_a][:num_questions]
    ground_truth_answers = [qa["answer"] for qa in q_a][:num_questions]
    generated_answers = []
    retrieved_documents = []

    # Generate answers and retrieve documents for each question
    for question in questions:
        context = retrieve_context_per_question(question, chunks_query_retriever)
        retrieved_documents.append(context)
        context_string = " ".join(context)
        result = answer_question_from_context(question, context_string, question_answer_from_context_chain)
        generated_answers.append(result["answer"])

    # Create test cases and evaluate
    test_cases = create_deep_eval_test_cases(questions, ground_truth_answers, generated_answers, retrieved_documents)
    evaluate(
        test_cases=test_cases,
        metrics=[correctness_metric, faithfulness_metric, relevance_metric]
    )

### Read PDF to String

In [85]:
%%time
content = read_pdf_to_string(path)
print_pretty(content)

ValueError: not enough values to unpack (expected 2, got 1)

### Breakpoint types

In [55]:
!pip install langchain_experimental -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 1.5 MB/s eta 0:00:00


In [81]:
from langchain_experimental.text_splitter import SemanticChunker

In [87]:
%%time
# chose which embeddings and breakpoint type and threshold to use
text_splitter = SemanticChunker(OpenAIEmbeddings(), breakpoint_threshold_type='percentile', breakpoint_threshold_amount=90)
print_pretty(text_splitter)



CPU times: user 132 ms, sys: 0 ns, total: 132 ms
Wall time: 130 ms


### Split original text to semantic chunks

In [88]:
%%time
docs = text_splitter.create_documents([content])
print_pretty(docs)


[Document(page_content="CHAPTER 1 \nUNIT - 1  \n1.MEANING AND SCOPE OF ACCOUNTING \n1.INTRODUCTION \nEvery individual performs some kind of economic activity. A salaried person gets \nsalary and spends to buy provisions and clothing, for children's education, \nconstruction of house, etc."), Document(page_content='A sports club formed by a group of individuals, a \nbusiness run by an individual or a group of individuals, a company running a \nbusiness in telecom sector, a local authority like Calcutta Municipal Corporation, \nDelhi Development Authority, Governments, either Central or State, all are \ncarrying some kind of economic activities. Not necessarily all the economic \nactivities are run for any individual benefit; such economic activities may create \nsocial benefit i.e.'), Document(page_content='benefit for the public, at large.'), Document(page_content='Anyway, such economic activities \nare performed through \'transactions and events\'. Transaction is used to mean \'a \nb

### Create vector store and retriever

In [ ]:
# embeddings = OpenAIEmbeddings()
# vectorstore = FAISS.from_documents(docs, embeddings)
# chunks_query_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [94]:
%%time
# OpenAI Embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    openai_api_key=OPENAI_API_KEY,
    dimensions=3072
  )
db = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="db"
)
chunks_query_retriever = db.as_retriever(search_kwargs={"k": 2})

print_pretty(chunks_query_retriever)


tags=['Chroma', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7d6334baab90> search_kwargs={'k': 2}

CPU times: user 8.51 s, sys: 187 ms, total: 8.69 s
Wall time: 13.9 s


### Test the retriever

In [95]:
%%time
query="What are the procedural aspects of accounting?"
result = retrieve_context_per_question(query, chunks_query_retriever)
print_pretty(result)


[Document(metadata={'page': 2, 'source': '/content/accounting.pdf'}, page_content='to assume a bigger responsibility than to merely do book-keeping. Accountants need to be ready to provide the information ready for the intended users to be able to make economic decisions. 2.1.Procedural aspects of Accounting On the basis of the above definitions, procedure of accounting can be basically divided into two parts: i.Generating financial information and ii.Using the financial information. Generating Financial Information 1.Recording - This is the basic function of accounting. All'), Document(metadata={'page': 2, 'source': '/content/accounting.pdf'}, page_content='to assume a bigger responsibility than to merely do book-keeping. Accountants need to be ready to provide the information ready for the intended users to be able to make economic decisions. 2.1.Procedural aspects of Accounting On the basis of the above definitions, procedure of accounting can be basically divided into two parts: i

### LLM

In [96]:
%%time
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from IPython.display import display, HTML

# OpenAI Embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    openai_api_key=OPENAI_API_KEY,
    dimensions=3072
)

# Create Chroma database
db = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="db"
)

# Create retriever
chunks_query_retriever = db.as_retriever(search_kwargs={"k": 2})

# Initialize LLM
llm = ChatOpenAI(
    name="gpt-4o",
    api_key=OPENAI_API_KEY,
    temperature=0
)

# Optimized prompt template
prompt_template = """
You are an AI assistant tasked with answering questions based solely on the provided context.
Your response should be derived entirely from the given information, without adding external knowledge.
Maintain the original tone and style of the source material, only improving readability and coherence where necessary.

Context:
{context}

Question: {question}

Instructions:
1. Use only the information provided in the context to answer the question.
2. If the context doesn't contain relevant information, state that you cannot answer based on the given context.
3. Preserve the key terminology and concepts from the original text.
4. Organize the information logically if it comes from multiple sources.
5. Do not introduce any new facts or interpretations beyond what's in the context.

Answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

# Create QA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chunks_query_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

CPU times: user 8.62 s, sys: 259 ms, total: 8.87 s
Wall time: 11.9 s


In [97]:
%%time
def get_answer(query):
    result = qa({"query": query})

    output = "<pre style='white-space: pre-wrap;'>"
    output += f"<b>Query:</b> {query}\n\n"
    output += f"<b>Answer:</b>\n{result['result']}\n\n"
    output += "<b>Source Documents:</b>\n"
    for i, doc in enumerate(result['source_documents'], 1):
        output += f"Document {i}:\n"
        output += f"Content: {doc.page_content}\n"
        output += f"Metadata: {doc.metadata}\n"
        output += "-" * 80 + "\n\n"
    output += "</pre>"

    display(HTML(output))

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10.3 µs


In [98]:
# Execute the query and display results
query = "What are the procedural aspects of accounting?"
get_answer(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
